# PalmRecognition
jupyter版本

### __INIT__.kaggle环境初始化命令

In [3]:
%%bash
rm -rf /kaggle/working
ln -s /kaggle/input/palmbigdatabase/PalmBigDataBase /kaggle/working

rm: cannot remove '/kaggle/working': Device or resource busy


### __INIT__.导入包

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm

### __INIT__.配置加载——从文件或手动配置

In [6]:
import yaml
try:
    with open("args.yml", "r", encoding="utf-8") as f:
        cfg = yaml.safe_load(f)
except:
    cfg = yaml.safe_load("""
model:
  name: noise_split_net

img_basic_info:
  img_height: 128
  img_width: 128
  img_channels: 3

TTA:
  enabled: true
  n_augmentations: 5

train:
  batch_size: 512
  epochs: 50
  learning_rate: 0.001
  weight_decay: 0.0001
  lr_step_size: 20
  lr_gamma: 0.5

test:
  batch_size: 32
  shuffle: false

dev:
  batch_size: 32
  shuffle: false""")

print(cfg["train"]["batch_size"])

512


In [7]:
# 预处理与 DataLoader
transform = T.Compose([
    T.Resize((cfg["img_basic_info"]["img_height"], cfg["img_basic_info"]["img_width"])),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5]),
])

## **MAIN**

### ::分类方法

In [ ]:
class PalmDataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None, cache=True):
        self.root = Path(root)
        self.samples = sorted(self.root.glob("*.bmp"))
        self.transform = transform
        self.target_transform = target_transform
        self.cache = cache
        self._cache_data = {}  # 缓存字典

        # 修改：区分左右手，格式 "F_100" 或 "S_100"
        ids = sorted({self._get_identity(p.name) for p in self.samples})
        self.id2idx = {pid: idx for idx, pid in enumerate(ids)}

        # 如果启用缓存，预加载所有图像
        if self.cache:
            print(f"正在缓存 {len(self.samples)} 张图像到内存...")
            for idx in tqdm(range(len(self.samples)), desc="加载图像"):
                path = self.samples[idx]
                with Image.open(path) as img:
                    img = img.convert("L")  # 灰度
                    self._cache_data[idx] = img.copy()  # 复制到内存
            print("缓存完成！")

    @staticmethod
    def _get_identity(filename):
        # 例：P_F_100_1.bmp → "F_100" (右手ID=100)
        # 例：P_S_100_1.bmp → "S_100" (左手ID=100)
        parts = filename.split("_")
        hand = parts[1]  # F 或 S
        person_id = parts[2]
        return f"{hand}_{person_id}"

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # 从缓存或磁盘读取图像
        if self.cache and index in self._cache_data:
            img = self._cache_data[index]
        else:
            path = self.samples[index]
            with Image.open(path) as img:
                img = img.convert("L")  # 灰度

        label = self.id2idx[self._get_identity(self.samples[index].name)]
        
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)

        return img, label

# 启用缓存（cache=True），如果内存不足可以设置为 False
dataset = PalmDataset("PalmBigDataBase", transform=transform, cache=True)

train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_set, val_set = random_split(dataset, [train_len, val_len])

正在缓存 7752 张图像到内存...


加载图像: 100%|██████████| 7752/7752 [01:30<00:00, 85.46it/s]

缓存完成！


In [ ]:
batch_size=1000
train_loader = DataLoader(train_set, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size, shuffle=False, num_workers=4)

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        # VGG-style convolutional blocks
        # Block 1: 128 -> 64
        self.conv1_1 = nn.Conv2d(1, 64, 3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Block 2: 64 -> 32
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Block 3: 32 -> 16
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Block 4: 16 -> 8
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        # 输入尺寸: 128 -> 64 -> 32 -> 16 -> 8, 最终为 512 * 8 * 8
        self.fc1 = nn.Linear(512 * 8 * 8, 4096)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 4096)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(4096, 386)

    def forward(self, x):
        # Block 1
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        
        # Block 2
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        
        # Block 3
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool3(x)
        
        # Block 4
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool4(x)
        
        # Flatten and fully connected
        x = x.view(-1, 512 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

In [60]:
def fit():
    model = Net().to("cuda")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    for epoch in range(200):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to("cuda"), labels.to("cuda")

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    print("Finished Training")

fit()

Epoch 1, Loss: 6.210393686445274
Epoch 2, Loss: 6.221557917700259
Epoch 2, Loss: 6.221557917700259
Epoch 3, Loss: 6.217576273793733
Epoch 3, Loss: 6.217576273793733
Epoch 4, Loss: 6.217259756270349
Epoch 4, Loss: 6.217259756270349
Epoch 5, Loss: 6.220301280231596
Epoch 5, Loss: 6.220301280231596
Epoch 6, Loss: 6.223497591947591
Epoch 6, Loss: 6.223497591947591
Epoch 7, Loss: 6.218075819043186
Epoch 7, Loss: 6.218075819043186


KeyboardInterrupt: 

### ::认证方法
模型用于提取特征，然后通过计算特征之间的距离来进行认证。

数据集5训练，5测试

#### 方法设计

##### 模型结构

In [8]:
class INet(nn.Module):
    def __init__(self, feature_dim=128):
        super(INet, self).__init__()
        # 特征提取网络
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)  # 128 -> 64
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)  # 64 -> 32
        
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)  # 32 -> 16
        
        # 全连接层
        self.fc1 = nn.Linear(256 * 16 * 16, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, feature_dim)
    
    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        # Block 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        # Block 3
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool3(x)
        
        # Flatten and FC
        x = x.view(-1, 256 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x

In [9]:
# 提取特征并进行认证
def get_pattern(model, imgs):
    """提取图像特征"""
    model.eval()
    device = next(model.parameters()).device
    with torch.no_grad():
        imgs = imgs.to(device)
        features = model(imgs)
        features = F.normalize(features, p=2, dim=1)  # L2归一化
    return features.cpu()

def authenticate(model, query_img, gallery_features, threshold=0.6):
    """
    认证功能：判断query_img是否属于gallery中的某个身份
    
    Args:
        model: 特征提取模型
        query_img: 查询图像 (1, C, H, W)
        gallery_features: 画廊特征库 (N, feature_dim)
        threshold: 认证阈值，距离小于此值认为匹配成功
    
    Returns:
        is_authenticated: 是否认证成功
        min_distance: 最小距离
        matched_idx: 匹配的索引
    """
    query_feature = get_pattern(model, query_img.unsqueeze(0))  # (1, feature_dim)
    
    # 计算与画廊中所有特征的距离
    distances = torch.cdist(query_feature, gallery_features).squeeze(0)  # (N,)
    min_distance, matched_idx = torch.min(distances, dim=0)
    
    is_authenticated = min_distance.item() < threshold
    
    return is_authenticated, min_distance.item(), matched_idx.item()

def build_gallery(model, dataloader):
    """构建特征画廊"""
    model.eval()
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in tqdm(dataloader, desc="构建特征画廊"):
            features = get_pattern(model, imgs)
            all_features.append(features)
            all_labels.append(labels)
    
    return torch.cat(all_features, dim=0), torch.cat(all_labels, dim=0)

# 评估认证性能
def evaluate_authentication(model, threshold=0.6):
    """评估认证模型的FAR和FRR"""
    device = next(model.parameters()).device
    
    # 构建画廊（使用训练集的平均特征）
    print("构建特征画廊...")
    gallery_features, gallery_labels = build_gallery(model, auth_train_loader)
    
    # 对每个身份计算平均特征
    unique_labels = torch.unique(gallery_labels)
    avg_features = []
    for label in unique_labels:
        mask = gallery_labels == label
        avg_feat = gallery_features[mask].mean(dim=0, keepdim=True)
        avg_features.append(avg_feat)
    gallery_features = torch.cat(avg_features, dim=0)  # (num_identities, feature_dim)
    
    # 在测试集上评估
    print(f"\n使用阈值 {threshold} 进行认证测试...")
    genuine_scores = []  # 真实匹配的距离
    impostor_scores = []  # 冒充者的距离
    
    for imgs, labels in tqdm(auth_test_loader, desc="认证测试"):
        imgs = imgs.to(device)
        features = get_pattern(model, imgs)
        
        for i, (feat, label) in enumerate(zip(features, labels)):
            # 计算与所有画廊特征的距离
            distances = torch.cdist(feat.unsqueeze(0), gallery_features).squeeze(0)
            min_dist, matched_idx = torch.min(distances, dim=0)
            
            if matched_idx.item() == label.item():
                genuine_scores.append(min_dist.item())
            else:
                impostor_scores.append(min_dist.item())
    
    # 计算FAR和FRR
    genuine_scores = torch.tensor(genuine_scores)
    impostor_scores = torch.tensor(impostor_scores)
    
    FRR = (genuine_scores > threshold).float().mean().item() * 100  # 误拒率
    FAR = (impostor_scores < threshold).float().mean().item() * 100  # 误识率
    
    print(f"\n认证性能评估:")
    print(f"阈值: {threshold}")
    print(f"FRR (误拒率): {FRR:.2f}%")
    print(f"FAR (误识率): {FAR:.2f}%")
    print(f"平均真实距离: {genuine_scores.mean():.4f}")
    print(f"平均冒充距离: {impostor_scores.mean():.4f}")
    
    return FRR, FAR

# 使用示例（先训练模型或加载已训练的模型）
# model = INet(feature_dim=128).to("cuda")
# model.load_state_dict(torch.load('best_auth_model.pth'))
# evaluate_authentication(model, threshold=0.6)

##### 使用交叉熵损失

In [17]:
from torch.utils.data import Dataset

# 构建认证数据集（每个人取5张训练，5张测试）
class AuthDataset(Dataset):
    def __init__(self, root, mode='train', transform=None):
        self.root = Path(root)
        self.mode = mode
        self.transform = transform
        
        # 按身份ID分组样本（区分左右手）
        all_samples = sorted(self.root.glob("*.bmp"))
        id_groups = {}
        for path in all_samples:
            pid = self._get_identity(path.name)
            if pid not in id_groups:
                id_groups[pid] = []
            id_groups[pid].append(path)
        
        # 每个ID取5张训练，5张测试
        self.samples = []
        for pid, paths in id_groups.items():
            if mode == 'train':
                self.samples.extend(paths[:5])
            else:  # test
                self.samples.extend(paths[5:10])
        
        self.samples = sorted(self.samples)
        ids = sorted({self._get_identity(p.name) for p in self.samples})
        self.id2idx = {pid: idx for idx, pid in enumerate(ids)}
    
    @staticmethod
    def _get_identity(filename):
        # 区分左右手：P_F_100_1.bmp → "F_100", P_S_100_1.bmp → "S_100"
        parts = filename.split("_")
        hand = parts[1]  # F 或 S
        person_id = parts[2]
        return f"{hand}_{person_id}"
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        path = self.samples[index]
        img = Image.open(path).convert("L")
        label = self.id2idx[self._get_identity(path.name)]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

# 创建认证数据集（Windows下 num_workers=0）
auth_train_set = AuthDataset("PalmBigDataBase", mode='train', transform=transform)
auth_test_set = AuthDataset("PalmBigDataBase", mode='test', transform=transform)

auth_train_loader = DataLoader(auth_train_set, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
auth_test_loader = DataLoader(auth_test_set, batch_size=64, shuffle=False, num_workers=0)

In [ ]:
# 训练函数（使用分类loss + 特征归一化）
def train_auth_model(epochs=50, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = INet(feature_dim=128).to(device)
    
    # 使用分类头进行训练
    num_classes = len(auth_train_set.id2idx)
    classifier = nn.Linear(128, num_classes).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(list(model.parameters()) + list(classifier.parameters()), 
                                   lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    
    best_acc = 0.0
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        classifier.train()
        running_loss = 0.0
        
        for inputs, labels in tqdm(auth_train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            features = model(inputs)
            features_norm = F.normalize(features, p=2, dim=1)  # L2归一化
            outputs = classifier(features_norm)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(auth_train_loader)
        scheduler.step()
        
        # 验证阶段
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in auth_test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                features = model(inputs)
                features_norm = F.normalize(features, p=2, dim=1)
                outputs = classifier(features_norm)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, Test Acc={acc:.2f}%")
        
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), 'best_auth_model.pth')
            print(f"✓ 保存最佳模型,准确率: {best_acc:.2f}%")
    
    print(f"\n训练完成！最佳准确率: {best_acc:.2f}%")
    return model

# 开始训练（先运行上面创建 auth_train_set 的单元格）
# model = train_auth_model(epochs=30, lr=0.001)

##### 使用对比损失

In [10]:
# 对比损失训练需要的数据集（返回样本对）
class ContrastivePairDataset(Dataset):
    def __init__(self, root, mode='train', transform=None):
        self.root = Path(root)
        self.mode = mode
        self.transform = transform
        
        # 按身份ID分组样本（区分左右手）
        all_samples = sorted(self.root.glob("*.bmp"))
        id_groups = {}
        for path in all_samples:
            pid = self._get_identity(path.name)
            if pid not in id_groups:
                id_groups[pid] = []
            id_groups[pid].append(path)
        
        # 每个ID取5张训练，5张测试
        self.id_groups = {}
        for pid, paths in id_groups.items():
            if mode == 'train':
                selected = paths[:5]
            else:  # test
                selected = paths[5:10]
            
            # 只保留有样本的身份
            if len(selected) > 0:
                self.id_groups[pid] = selected
        
        self.ids = list(self.id_groups.keys())
    
    @staticmethod
    def _get_identity(filename):
        # 区分左右手：P_F_100_1.bmp → "F_100", P_S_100_1.bmp → "S_100"
        parts = filename.split("_")
        hand = parts[1]  # F 或 S
        person_id = parts[2]
        return f"{hand}_{person_id}"
    
    def __len__(self):
        return len(self.ids) * 10  # 每个ID生成10对样本
    
    def __getitem__(self, index):
        import random
        
        # 50%正样本对，50%负样本对
        if index % 2 == 0:
            # 正样本对（同一个人同一只手）
            pid = self.ids[index // 10 % len(self.ids)]
            paths = self.id_groups[pid]
            if len(paths) >= 2:
                # 随机选择两张不同的图片
                path1, path2 = random.sample(paths, 2)
            else:
                path1 = path2 = paths[0]
            label = 1.0
        else:
            # 负样本对（不同的人或不同的手）
            idx1 = (index // 10) % len(self.ids)
            idx2 = (idx1 + 1 + random.randint(0, len(self.ids) - 2)) % len(self.ids)
            
            pid1 = self.ids[idx1]
            pid2 = self.ids[idx2]
            
            path1 = random.choice(self.id_groups[pid1])
            path2 = random.choice(self.id_groups[pid2])
            label = 0.0
        
        img1 = Image.open(path1).convert("L")
        img2 = Image.open(path2).convert("L")
        
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return img1, img2, torch.tensor(label, dtype=torch.float32)

# 创建对比损失数据集
contrastive_train_set = ContrastivePairDataset("PalmBigDataBase", mode='train', transform=transform)
contrastive_test_set = ContrastivePairDataset("PalmBigDataBase", mode='test', transform=transform)

contrastive_train_loader = DataLoader(contrastive_train_set, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
contrastive_test_loader = DataLoader(contrastive_test_set, batch_size=32, shuffle=False, num_workers=0)

In [11]:
contrastive_test_set[0][0].shape

torch.Size([1, 128, 128])

In [23]:
# 对比损失函数（基于余弦相似度的改进版本）
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=0.5):
        """
        Args:
            margin: 余弦距离的margin，范围[0, 2]
                   对于余弦相似度[-1, 1]，转换为余弦距离[0, 2]
        """
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    
    def forward(self, feature1, feature2, label):
        # label: 1表示相同身份，0表示不同身份
        # 先进行L2归一化
        feature1 = F.normalize(feature1, p=2, dim=1)
        feature2 = F.normalize(feature2, p=2, dim=1)
        
        # 计算余弦相似度
        cosine_similarity = F.cosine_similarity(feature1, feature2)
        
        # 转换为余弦距离 [0, 2]
        cosine_distance = 1 - cosine_similarity
        
        # 对比损失
        loss = torch.mean(
            label * torch.pow(cosine_distance, 2) +
            (1 - label) * torch.pow(torch.clamp(self.margin - cosine_distance, min=0.0), 2)
        )
        return loss

In [24]:
# 使用对比损失训练（基于余弦相似度）
def train_with_contrastive_loss(epochs=50, lr=0.001, margin=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = INet(feature_dim=128).to(device)
    
    criterion = ContrastiveLoss(margin=margin)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    
    best_loss = float('inf')
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        running_loss = 0.0
        
        for img1, img2, labels in tqdm(contrastive_train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # 提取特征
            features1 = model(img1)
            features2 = model(img2)
            
            # 计算对比损失（内部会进行归一化和余弦相似度计算）
            loss = criterion(features1, features2, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(contrastive_train_loader)
        scheduler.step()
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for img1, img2, labels in contrastive_test_loader:
                img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
                
                features1 = model(img1)
                features2 = model(img2)
                
                loss = criterion(features1, features2, labels)
                val_loss += loss.item()
                
                # 计算准确率（使用余弦距离）
                features1_norm = F.normalize(features1, p=2, dim=1)
                features2_norm = F.normalize(features2, p=2, dim=1)
                cosine_similarity = F.cosine_similarity(features1_norm, features2_norm)
                cosine_distance = 1 - cosine_similarity
                
                threshold = margin / 2
                predictions = (cosine_distance < threshold).float()
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        
        val_loss /= len(contrastive_test_loader)
        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Train Loss={avg_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.2f}%")
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'best_contrastive_model.pth')
            print(f"✓ 保存最佳模型，验证损失: {best_loss:.4f}, 准确率: {val_acc:.2f}%")
    
    print(f"\n训练完成！最佳验证损失: {best_loss:.4f}")
    return model

# 开始训练（margin设置为0.5，适合余弦距离[0,2]）
# model = train_with_contrastive_loss(epochs=30, lr=0.001, margin=0.5)

In [ ]:
model = train_with_contrastive_loss(epochs=100, lr=0.001, margin=0.5)

Epoch 1/30: 100%|██████████| 242/242 [01:02<00:00,  3.87it/s]



Epoch 1: Train Loss=0.0614, Val Loss=0.0747, Val Acc=62.54%
✓ 保存最佳模型，验证损失: 0.0747, 准确率: 62.54%


Epoch 2/30: 100%|██████████| 242/242 [01:03<00:00,  3.83it/s]


Epoch 2: Train Loss=0.0471, Val Loss=0.0563, Val Acc=69.38%
✓ 保存最佳模型，验证损失: 0.0563, 准确率: 69.38%


Epoch 3/30: 100%|██████████| 242/242 [01:03<00:00,  3.81it/s]


...

Epoch 28: Train Loss=0.0070, Val Loss=0.0048, Val Acc=97.24%
✓ 保存最佳模型，验证损失: 0.0048, 准确率: 97.24%


Epoch 29/30: 100%|██████████| 242/242 [01:01<00:00,  3.95it/s]


Epoch 29: Train Loss=0.0070, Val Loss=0.0045, Val Acc=97.74%
✓ 保存最佳模型，验证损失: 0.0045, 准确率: 97.74%


Epoch 30/30: 100%|██████████| 242/242 [01:01<00:00,  3.94it/s]


Epoch 30: Train Loss=0.0067, Val Loss=0.0052, Val Acc=97.06%

训练完成！最佳验证损失: 0.0045


In [26]:
!ls

best_contrastive_model.pth  PalmBigDataBase


: 

In [22]:
# 测试模型性能（复用 evaluate_authentication 函数）
def test_model(model_path='best_contrastive_model.pth', threshold=0.6):
    """
    测试训练好的模型性能 - 直接复用 evaluate_authentication 函数
    
    Args:
        model_path: 模型权重文件路径（可以是任何训练方法得到的模型）
                   - 'best_contrastive_model.pth': 对比损失训练的模型
                   - 'best_auth_model.pth': 交叉熵损失训练的模型
        threshold: 认证阈值
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 加载模型
    model = INet(feature_dim=128).to(device)
    model.load_state_dict(torch.load(model_path))
    
    print(f"正在测试模型: {model_path}")
    print("="*60)
    
    # 直接使用 evaluate_authentication 函数
    FRR, FAR = evaluate_authentication(model, threshold=threshold)
    
    return FRR, FAR

# 使用示例：
# 测试对比损失训练的模型
# test_model('best_contrastive_model.pth')

# # 测试交叉熵损失训练的模型
# test_model('best_auth_model.pth', threshold=0.6)

# 对比不同阈值的效果
for thresh in [0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f"\n{'='*60}")
    test_model('best_contrastive_model.pth', threshold=thresh)


正在测试模型: best_contrastive_model.pth
构建特征画廊...
正在测试模型: best_contrastive_model.pth
构建特征画廊...


构建特征画廊: 100%|██████████| 61/61 [00:11<00:00,  5.37it/s]



使用阈值 0.3 进行认证测试...


认证测试: 100%|██████████| 61/61 [00:12<00:00,  4.84it/s]




认证性能评估:
阈值: 0.3
FRR (误拒率): 0.00%
FAR (误识率): 100.00%
平均真实距离: 0.0000
平均冒充距离: 0.0000

正在测试模型: best_contrastive_model.pth
构建特征画廊...
正在测试模型: best_contrastive_model.pth
构建特征画廊...


构建特征画廊:  89%|████████▊ | 54/61 [00:10<00:01,  5.18it/s]


KeyboardInterrupt: 

##### 使用三元组损失

In [ ]:
# 使用Triplet Loss训练
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        pos_dist = F.pairwise_distance(anchor, positive)
        neg_dist = F.pairwise_distance(anchor, negative)
        loss = torch.mean(torch.clamp(pos_dist - neg_dist + self.margin, min=0.0))
        return loss

In [24]:
auth_train_set[0][0][0].mean()

tensor(0.0868)